In [1]:
#Total precipitation amount including only precipitation values that are greater than the nth percentile of precip 
#on wet days (precip > 1mm)

#Open the file with the desired station

#-------------------------------------------------------------
#FINDING THE PERCENTILE (1981-2010)

#
#Loop through each year:

#    within the year find all of the lines of data that hold precipitation information
#    Variable in question :: PRCP
#    Create a new array filled with the lines of data including PRCP values
#    Loop through each line:

#    If the value is coded as missing (PRCP = 9999) skip this value
#    If the value is < 1mm skip this value
#    If the value is > 1mm add it to the new array called 'total' and move on to the next value

#  Move onto the next year continuously adding to the 'total' array

#  Order the array with all the values > 1mm in order from smallest to largest
#  Multiply the number of values in the array (n) by the percentile desired (.99)
#    New variable for the 99th percentile = RRwn
#  Find the 99th percentile value: total[99n] = 99p

#---------------------------------------------------------------
#Trend of precipitation events above the nth percentile:
#For any given year find the number of days where the precipitation amount is above the 99th percentile (1961-1990)

#Loop through the precipitation data and count days where it's greater than the desired percentile
#Plot the resulting graph for each station

#---------------------------------------------------------------
#Create a text file with the 95th, 98th, and 99th percentile days
#The text file also includes the annual accumulated precip from days above the percentile
#Also saves as a csv file with all of the climate metrics

#----------------------------------------------------------------
#Use model ensembles to extend the data to 2100


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
import csv
import time
from netCDF4 import Dataset
from geopy.distance import great_circle

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#Opening a GHCN-D file and reading variables
#Lincoln Station = USW00014939
#Asheville: USW00003812
# Omaha:'USW00014942'

station = 'USW00014935'     #Goodland
 
filename = '/Users/rphinney/Documents/Hollings/ghcnd_all/'+station+'.dly'
file = open(filename, 'r')

station_array = ['USW00014942','USW00003812','USW00014939']

In [ ]:
#Loop through all stations

start = time.time()

midstations = open('/Users/rphinney/Documents/midstations.txt','r')
station_array = []

for n in midstations:
    all_id = n[0:11]
    station_array.append(all_id)

for station in station_array:
    station = station.strip()

    
    filename = '/Users/rphinney/Documents/Hollings/ghcnd_all/'+station+'.dly'
    file = open(filename, 'r')
    
    #Using ghcnd-stations.txt get the station information

    path = '/Users/rphinney/Documents/Hollings/ghcnd-stations.txt'

    station_file = open(path)
    ghcnd = []

    for x in station_file:
        ghcnd.append(x)

    for each_line in ghcnd:   
        if each_line[0:11] == station:
            station_name = each_line[40:70]
            station_state = each_line[38:40]
            latitude = each_line[13:20]
            longitude = each_line[21:31]
            elevation = each_line[32:38]
            station_name.strip()
            print(station_name)
            #print(type(latitude))
            #print(longitude)
            #print(elevation)
    #---------------------------------------------------------------------------------------------------        
    #create an array (mylist) filled with each line of data
    #the format type of the array is string so the numbers will need to be converted to integers later on

    mylist = []

    for line in file:
        mylist.append(line)

    #define our range of years for normals
    begin_year = 1980
    end_year = 2010


    all_prcp = []
    for year in range(begin_year, end_year):
        year_prcp = []
        #all_prcp = []
        for line in mylist:
            if int(line[11:15]) == year:
                if line[17:21] == 'PRCP':
                    year_prcp.append(line)

        for data in year_prcp:
            char1 = 22
            char2 = 26
            while char1 != 270:
                if data[char1:char2] == '9999':
                    char1 = char1 + 8
                    char2 = char2 + 8
                elif int(data[char1:char2]) < 1.:
                    char1 = char1 + 8
                    char2 = char2 + 8
                else:
                    prcp = (.003937*int(data[char1:char2]))      #convert to inches
                    all_prcp.append(prcp)
                    char1 = char1 + 8
                    char2 = char2 + 8
                    
    all_prcp.extend(all_prcp)  
        

    print(size(all_prcp))

    #----------------------------------------------------------------------------------------------------
    # sort the data from smallest value to largest
    all_prcp = sorted(all_prcp)

    # find the percentile location
    n_elements = size(all_prcp)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #If the historical data doesn't include any precipitation data write the file with the model data
    if n_elements == 0:
        
         #Model Data
    
        rcp = '4'

        filedays = '/Users/rphinney/Documents/Hollings/pr-days-above-99th/rcp'+rcp+'5/ensemble.nc'
        filepr = '/Users/rphinney/Documents/Hollings/pr-above-99th/rcp'+rcp+'5/ensemble.nc'
        filewd = '/Users/rphinney/Documents/Hollings/consecWD/rcp'+rcp+'5/ensemble.nc'
        filedd = '/Users/rphinney/Documents/Hollings/consecDD/rcp'+rcp+'5/ensemble.nc'
        filemax1 = '/Users/rphinney/Documents/Hollings/prmax1day/rcp'+rcp+'5/ensemble.nc'
        filemax5 = '/Users/rphinney/Documents/Hollings/prmax5day/rcp'+rcp+'5/ensemble.nc'
        data = Dataset(filedays)
        datapr = Dataset(filepr)
        datawd = Dataset(filewd)
        datadd = Dataset(filedd)
        datamax1 = Dataset(filemax1)
        datamax5 = Dataset(filemax5)

        lon = data.variables['lon'][:]
        lat = data.variables['lat'][:]
        print(data.variables.keys())
        days = data.variables['pr_days_above_99th'][:]
        pr = datapr.variables['pr_above_99th'][:]
        wd = datawd.variables['consecWD'][:]
        dd = datadd.variables['consecDD'][:]
        max1 = datamax1.variables['prmax1day'][:]
        max5 = datamax5.variables['prmax5day'][:]



        station_loc = (latitude,longitude)
        min_loc = 9999

        for lons in lon:
            for lats in lat:
                check_loc = (lats,lons)
                b = int(great_circle(station_loc,check_loc).miles)
                if b < min_loc:
                    min_loc = b
                    min_lat = lats
                    min_lon = lons-360

        print(pr[4,125,244])
        pra = []
        wda = []
        dda = []
        max1a = []
        max5a = []
        day1 = []
        day2 = []
        ann1 = []
        ann2 = []
        daysa = []
        lon_count = -1
        lat_count = -1
        for lons in lon:
            lon_count = lon_count +1
            if lons == min_lon+360:
                for lats in lat:
                    lat_count = lat_count + 1
                    if lats == min_lat:
                        for x in range (11,95):
                            prcp2 = days[x,lat_count,lon_count]
                            try: 
                                daysa.append(int(prcp2))
                            except:
                                pass
                            pr1 = .03937*(pr[x,lat_count,lon_count])
                            try:
                                pra.append(round(pr1,2))
                            except:
                                pass
                            wd1 = wd[x,lat_count,lon_count]
                            try:
                                wda.append(int(wd1))
                            except:
                                pass
                            dd1 = dd[x,lat_count,lon_count]
                            try:
                                dda.append(int(dd1))
                            except:
                                pass
                            max11 = .03937*(max1[x,lat_count,lon_count])
                            try:
                                max1a.append(round(max11,2))
                            except:
                                pass
                            max51 = .03927*(max5[x,lat_count,lon_count])
                            try:
                                max5a.append(round(max51,2))
                            except:
                                pass
                            day = 'N/A'
                            day1.append(day)
                            day2.append(day)
                            ann1.append(day)
                            ann2.append(day)

        
        
        year = np.arange(2006,2101)
    
        #plotting to csv file
        #create arrays for repeated variables:
        lat = []
        station_id = []
        lon = []
        ele = []
        stat_name = []
        for x in range(2101-2006):
            lat.append(latitude)
            lon.append(longitude)
            station_id.append(station)
            ele.append(elevation)
            stat_name.append(station_name)

        path = '/Users/rphinney/Documents/Hollings/csv'+rcp+'5/'
        file_id = open(path+station+'.csv', 'w')
        with open(path+station+'.csv', 'w') as csvfile:
            writerh = csv.DictWriter(csvfile, fieldnames = ['Station ID', 'Latitude', 'Longitude', 'Elevation', 'Station Name','Year', 'Days above 95%', 'Annual Precip Accum (in)','Days above 98%', 'Annual Precip Accum 98% (in)','Days above 99%', 'Annual Precip Accum 99% (in)','Max One Day Precip','Max 5 Day Precip Accum','Max Consecutive Wet Days','Max Consecutive Dry Days'])
            writerh.writeheader()

            writer = csv.writer(csvfile)
            writer.writerows(zip(station_id,lat,lon,ele,stat_name,year,day1,ann1,day2,ann2,daysa,pra,max1a,max5a,wda,dda))
            
        csvfile.close()
        continue
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #Variable can be changed depending on which percentile is being used
    percentile = .99
    loc = int(percentile*n_elements)    #int() vs round()
    print(loc)

    #find the precipitation value at the percentile location
    RRwn = (all_prcp[loc])
    print(RRwn)

    
    #-----------------------------------------------------------------------------------------------------
    #plots the number of days in which the precipitation amount was greater than the nth percentile for a given year
    #plots the annual amount of precipitation when the daily amount is greater than the nth percentile

    days = []
    annual = []

    #find the beginning year of the file

    for first in mylist:
        if first[17:21] == "PRCP":
            min_year = int(first[11:15])
            break

    #print(min_year)
    max_year = 2017
    #day_max = 0
    day_maxa = []      #array of max precip for one day each year
    five_max = []
    max_dry = []
    max_wet = []
    for year in range(min_year,max_year):
        year_prcp = []
        for line in mylist:
            if int(line[11:15]) == year:
                if line[17:21] == 'PRCP':
                    year_prcp.append(line)

        new_prcp = []
        trace = []
        for data in year_prcp:
            char1 = 22
            char2 = 26
            while char1 != 270:
                if data[char1:char2] == '9999':
                    char1 = char1 + 8
                    char2 = char2 + 8
                else:
                    prcp = (.003937*int(data[char1:char2])) 
                    new_prcp.append(round(prcp,2))
                    mflag = data[char2:char2+1]
                    trace.append(mflag)
                    char1 = char1 + 8
                    char2 = char2 + 8
                    

        #Get the maximum one-day precip value------------------------------------------
        day_max = max(new_prcp, default = 0)
        if day_max == 0:
            day_max = 'N/A'
        day_maxa.append(day_max)    
        
        #Get the maximum five-day precip value-----------------------------------------
        begin = 0
        end = 5
        max5 = sum(new_prcp[begin:end])
    
        for x in new_prcp:
            begin = begin + 1
            end = end + 1
            b = sum(new_prcp[begin:end])
            max5 = max(b, max5)
        if max5 == 0:
            max5 = 'N/A'
        five_max.append(max5)            
                   
        #Count the days above the percentile-------------------------------------------            
        day_count = 0  
        annprcp = 0
        for one in new_prcp:
            if one > RRwn:
                day_count = 1 + day_count
                annprcp = one + annprcp
                
                
        #Annual max consecutive wet days-----------------------------------------------
        count_wet = 0
        wet_days = 0
        num = 0
        
        for a in range(0,size(new_prcp)):
            value_w = new_prcp[a]
            #trace = trace[num]
            if value_w > 0.0:
                count_wet = count_wet + 1
            if value_w == 0:
                if trace[a] == 'T':
                    count_wet = count_wet + 1
                else:
                    wet_days = max(count_wet, wet_days)
                    count_wet = 0
        if wet_days == 0:
            wet_days = 'N/A'
              
        max_wet.append(wet_days)  
        
        #Annal max consecutive dry days-------------------------------------------------
        count_dry = 0
        dry_days = 0
        
        for b in range(0,size(new_prcp)):
            value_d = new_prcp[b]
            if value_d == 0:
                if trace[b] == 'T':
                    dry_days = max(count_dry, dry_days)
                    count_dry = 0
                else:
                    count_dry = count_dry + 1
            if value_d > 0:
                dry_days = max(count_dry, dry_days)
                count_dry = 0           
        if dry_days == 0:
            dry_days = 'N/A'
        max_dry.append(dry_days)

        days.append(day_count)
        annual.append(annprcp)

        #print(year_prcp)
        
        
    #----------------------------------------------------------------------------------------------------
    #Plotting
    
    year = np.arange(min_year, 2017)
    plt.figure(figsize(15, 9))
    plt.bar(year, days, color = 'cadetblue')#, align = 'center')

    plt.suptitle ('Number of days above the ' + str(int(percentile*100))+ 'th percentile (1980-2010 normals)', size = 16)
    plt.title(station_name.strip()+ ', '+station_state, size = 13)
    plt.ylabel('Days', size = 11)
    if size(year) < 25:
        plt.xticks(year)
    plt.figtext(.13, .884, str(int(percentile*100))+'th Percentile Precipitation amount: '+ str(round(RRwn,2)) + ' in')

    savefig('/Users/rphinney/Documents/Hollings/day_plots/'+station+'.png')

    year = np.arange(min_year, max_year)
    #plt.plot(year, annual)
    #plt.suptitle('Total Annual Precipitation Accumulation when Daily Precipitation is Above the ' +str(int(percentile*100))+'th Percentile', size = 15)
    #plt.ylabel('Precipitation (in)')
    #plt.title(station_name.strip()+', '+station_state, size = 12)

    #-----------------------------------------------------------------------------------------------------
    #Get the 95th percentile

    # find the percentile location
    n_elements = size(all_prcp)
    percentile = .95
    loc = int(percentile*n_elements)
    RRwn = (all_prcp[loc])
    days1 = []
    annual1 = []
    day_c1 = []
    ann1 = []

    #find the beginning year of the file
    for first in mylist:
        if first[17:21] == "PRCP":
            min_year = int(first[11:15])
            break
    max_year = 2017



    for year in range(min_year,max_year):
        year_prcp = []
        for line in mylist:
            if int(line[11:15]) == year:
                if line[17:21] == 'PRCP':
                    year_prcp.append(line)

        new_prcp = []
        for data in year_prcp:
            char1 = 22
            char2 = 26
            while char1 != 270:
                if data[char1:char2] == '9999':
                    char1 = char1 + 8
                    char2 = char2 + 8
                else:
                    prcp = (.003937*int(data[char1:char2])) 
                    new_prcp.append(prcp)
                    char1 = char1 + 8
                    char2 = char2 + 8

        #array for the csv file (prints N/A for the precip values when there are no days above the percentile)           
        day_count = 0  
        annprcp = 0
        for one in new_prcp:
            if one > RRwn:
                day_count = 1 + day_count
                annprcp = one + annprcp
                annprcp = round(annprcp,2)
       
        if annprcp == 0:
            annprcp = 'N/A'

        days1.append(day_count)
        annual1.append(annprcp)
        
        
        #array for the txt file
        day_c = 0
        prcp_c = 0
        for one in new_prcp:
            if one > RRwn:
                day_c = 1 + day_c
                prcp_c = one + prcp_c
                prcp_c = round(prcp_c,2)   
        
        day_c1.append(day_c)
        ann1.append(prcp_c)
        


    #--------------------------------------------------------------------------------------------------
    #Get the 98th percentile

    # find the percentile location
    n_elements = size(all_prcp)
    percentile = .98
    loc = int(percentile*n_elements)
    RRwn = (all_prcp[loc])
    days2 = []
    annual2 = []
    day_c2 = []
    ann2 = []

    #find the beginning year of the file
    for first in mylist:
        if first[17:21] == "PRCP":
            min_year = int(first[11:15])
            break
    max_year = 2017

    
    for year in range(min_year,max_year):
        year_prcp = []
        for line in mylist:
            if int(line[11:15]) == year:
                if line[17:21] == 'PRCP':
                    year_prcp.append(line)

        new_prcp = []
        for data in year_prcp:
            char1 = 22
            char2 = 26
            while char1 != 270:
                if data[char1:char2] == '9999':
                    char1 = char1 + 8
                    char2 = char2 + 8
                else:
                    prcp = (.003937*int(data[char1:char2])) 
                    new_prcp.append(prcp)
                    char1 = char1 + 8
                    char2 = char2 + 8

        #array for the csv file (prints N/A for the precip values when there are no days above the percentile)           
        day_count = 0  
        annprcp = 0
        for one in new_prcp:
            if one > RRwn:
                day_count = 1 + day_count
                annprcp = one + annprcp
                annprcp = round(annprcp,2)

        if annprcp == 0:
            annprcp = 'N/A'
            
        days2.append(day_count)
        annual2.append(annprcp)
        
        
        #array for the txt file
        day_c = 0
        prcp_c = 0
        for one in new_prcp:
            if one > RRwn:
                day_c = 1 + day_c
                prcp_c = one + prcp_c
                prcp_c = round(prcp_c,2)   
        
        day_c2.append(day_c)
        ann2.append(prcp_c)


    #---------------------------------------------------------------------------------------------------------
    #Find the 99th percentile

    # find the percentile location
    n_elements = size(all_prcp)
    percentile = .99
    loc = int(percentile*n_elements)
    RRwn = (all_prcp[loc])
    days3 = []
    annual3 = []
    day_c3 = []
    ann3 = []

    #find the beginning year of the file
    for first in mylist:
        if first[17:21] == "PRCP":
            min_year = int(first[11:15])
            break
    max_year = 2017



    for year in range(min_year,max_year):
        year_prcp = []
        for line in mylist:
            if int(line[11:15]) == year:
                if line[17:21] == 'PRCP':
                    year_prcp.append(line)

        new_prcp = []
        for data in year_prcp:
            char1 = 22
            char2 = 26
            while char1 != 270:
                if data[char1:char2] == '9999':
                    char1 = char1 + 8
                    char2 = char2 + 8
                else:
                    prcp = (.003937*int(data[char1:char2])) 
                    new_prcp.append(prcp)
                    char1 = char1 + 8
                    char2 = char2 + 8
        
        
        #array for the csv file (prints N/A for the precip values when there are no days above the percentile)
        day_count = 0  
        annprcp = 0
        for one in new_prcp:
            if one > RRwn:
                day_count = 1 + day_count
                annprcp = one + annprcp
                annprcp = round(annprcp,2)
        if annprcp == 0:
            annprcp = 'N/A'
            

        days3.append(day_count)
        annual3.append(annprcp)
        
        
        #array for the txt file 
        day_c = 0
        prcp_c = 0
        for one in new_prcp:
            if one > RRwn:
                day_c = 1 + day_c
                prcp_c = one + prcp_c
                prcp_c = round(prcp_c,2)   
        
        day_c3.append(day_c)
        ann3.append(prcp_c)
        

        
    #----------------------------------------------------------------------------------------------------
    #Model Data
    
    rcp = '4'
    
    filedays = '/Users/rphinney/Documents/Hollings/pr-days-above-99th/rcp'+rcp+'5/ensemble.nc'
    filepr = '/Users/rphinney/Documents/Hollings/pr-above-99th/rcp'+rcp+'5/ensemble.nc'
    filewd = '/Users/rphinney/Documents/Hollings/consecWD/rcp'+rcp+'5/ensemble.nc'
    filedd = '/Users/rphinney/Documents/Hollings/consecDD/rcp'+rcp+'5/ensemble.nc'
    filemax1 = '/Users/rphinney/Documents/Hollings/prmax1day/rcp'+rcp+'5/ensemble.nc'
    filemax5 = '/Users/rphinney/Documents/Hollings/prmax5day/rcp'+rcp+'5/ensemble.nc'
    data = Dataset(filedays)
    datapr = Dataset(filepr)
    datawd = Dataset(filewd)
    datadd = Dataset(filedd)
    datamax1 = Dataset(filemax1)
    datamax5 = Dataset(filemax5)

    lon = data.variables['lon'][:]
    lat = data.variables['lat'][:]
    print(data.variables.keys())
    days = data.variables['pr_days_above_99th'][:]
    pr = datapr.variables['pr_above_99th'][:]
    wd = datawd.variables['consecWD'][:]
    dd = datadd.variables['consecDD'][:]
    max1 = datamax1.variables['prmax1day'][:]
    max5 = datamax5.variables['prmax5day'][:]
    


    station_loc = (latitude,longitude)
    min_loc = 9999

    for lons in lon:
        for lats in lat:
            check_loc = (lats,lons)
            b = int(great_circle(station_loc,check_loc).miles)
            if b < min_loc:
                min_loc = b
                min_lat = lats
                min_lon = lons-360
     
    print(pr[4,125,244])
    pra = []
    wda = []
    dda = []
    max1a = []
    max5a = []
    day1 = []
    day2 = []
    ann1 = []
    ann2 = []
    daysa = []
    lon_count = -1
    lat_count = -1
    for lons in lon:
        lon_count = lon_count +1
        if lons == min_lon+360:
            for lats in lat:
                lat_count = lat_count + 1
                if lats == min_lat:
                    for x in range (11,95):
                        prcp2 = days[x,lat_count,lon_count]
                        try: 
                            daysa.append(int(prcp2))
                        except:
                            pass
                        pr1 = .03937*(pr[x,lat_count,lon_count])
                        try:
                            pra.append(round(pr1,2))
                        except:
                            pass
                        wd1 = wd[x,lat_count,lon_count]
                        try:
                            wda.append(int(wd1))
                        except:
                            pass
                        dd1 = dd[x,lat_count,lon_count]
                        try:
                            dda.append(int(dd1))
                        except:
                            pass
                        max11 = .03937*(max1[x,lat_count,lon_count])
                        try:
                            max1a.append(round(max11,2))
                        except:
                            pass
                        max51 = .03927*(max5[x,lat_count,lon_count])
                        try:
                            max5a.append(round(max51,2))
                        except:
                            pass
                        day = 'N/A'
                        day1.append(day)
                        day2.append(day)
                        ann1.append(day)
                        ann2.append(day)

    
    days3.extend(daysa)
    annual3.extend(pra)
    day_maxa.extend(max1a)
    five_max.extend(max5a)
    max_wet.extend(wda)
    max_dry.extend(dda)
    days1.extend(day1)
    days2.extend(day2)
    annual1.extend(ann1)
    annual2.extend(ann2)
        
    #-----------------------------------------------------------------------------------------------------
    year = np.arange(min_year,2101)
    #export the data from the first graph into a text file
    

    
    
    #plotting to csv file
    
    #create arrays for repeated variables:
    lat = []
    station_id = []
    lon = []
    ele = []
    stat_name = []
    for x in range(2101-min_year):
        lat.append(latitude)
        lon.append(longitude)
        station_id.append(station)
        ele.append(elevation)
        stat_name.append(station_name)
        
    path = '/Users/rphinney/Documents/Hollings/csv'+rcp+'5/'
    file_id = open(path+station+'.csv', 'w')
    with open(path+station+'.csv', 'w') as csvfile:
        writerh = csv.DictWriter(csvfile, fieldnames = ['Station ID', 'Latitude', 'Longitude', 'Elevation', 'Station Name','Year', 'Days above 95%', 'Annual Precip Accum (in)','Days above 98%', 'Annual Precip Accum 98% (in)','Days above 99%', 'Annual Precip Accum 99% (in)','Max One Day Precip','Max 5 Day Precip Accum','Max Consecutive Wet Days','Max Consecutive Dry Days'])
        writerh.writeheader()
        
        writer = csv.writer(csvfile)
        writer.writerows(zip(station_id,lat,lon,ele,stat_name,year,days1,annual1,days2,annual2,days3,annual3,day_maxa,five_max,max_wet,max_dry))

end = time.time()
print(end-start)

 KEY WEST                     
0
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 MIAMI WSO CITY               
5348
5241
2.5511760000000003
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 BURRWOOD WB                  
0
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 COCOA BEACH PATRICK AFB      
0
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 CAPE KENNEDY AFS             
0
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 SARASOTA BRADENTON AP        
2536
2485
2.55905
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 ST PETERSBURG INTL AP        
2716
2661
2.389759
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat'

/Users/rphinney/anaconda/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 CORPUS CHRISTI               
4572
4480
2.669286
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 BEEVILLE CHASE NAAS          
2154
2110
2.279523
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 CORPUS CHRISTI NAS           
3744
3669
3.051175
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 HOUMA TERREBONNE AP          
0
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 KINGSVILLE NAAS              
4282
4196
2.338578
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'lat', 'lat_bounds', 'pr_days_above_99th'])
86.9561
 NEW ORLEANS AUDUBON          
6982
6842
2.9999940000000005
odict_keys(['time', 'time_bounds', 'lon', 'lon_bounds', 'la

In [ ]:
#print(year_prcp)

In [ ]:
#print(trace)